# TheBrain: PyTorch AI Model Training & Implementation Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meahmedrabbi/TheBrain/blob/main/TheBrain_PyTorch_Colab.ipynb)

This notebook demonstrates how to train and use TheBrain, a conversational AI model built with PyTorch. The model supports:
- **Transformer architecture** (attention-based, better for longer contexts)
- **LSTM architecture** (recurrent, good for smaller datasets)
- **Multiple open datasets** for training
- **Easy deployment** and inference

---

## 1. Setup & Installation

First, let's set up the environment and install required packages.

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tqdm numpy

In [ ]:
# Clone the repository
!git clone https://github.com/meahmedrabbi/TheBrain.git
%cd TheBrain

In [ ]:
# Check GPU availability
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Download and Prepare Multiple Datasets

We'll use several open-source conversational datasets:
1. **Cornell Movie Dialogs** - Movie character conversations
2. **DailyDialog** - Multi-turn daily conversations
3. **Custom conversational data** - Included in the repo

In [ ]:
# Download Cornell Movie Dialogs dataset
!wget -q http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -q cornell_movie_dialogs_corpus.zip
!mv cornell\ movie-dialogs\ corpus cornell_movie_dialogs
print("✓ Cornell Movie Dialogs downloaded")

In [ ]:
# Download DailyDialog dataset (sample)
# Note: Full dataset requires registration, so we'll create a sample
import os

# Create sample DailyDialog-style data
sample_dialogs = [
    "Good morning! How are you today? __eou__ I'm doing great, thank you! How about you? __eou__ I'm wonderful, thanks for asking.",
    "What's your favorite food? __eou__ I love pizza! What about you? __eou__ I prefer pasta.",
    "Have you seen the new movie? __eou__ Yes, I watched it last weekend. It was amazing! __eou__ I heard great reviews about it.",
    "What are your plans for the weekend? __eou__ I'm going hiking with friends. __eou__ That sounds fun!",
    "How was your day? __eou__ It was quite busy but productive. __eou__ I'm glad to hear that."
]

os.makedirs('data', exist_ok=True)
with open('data/sample_dailydialog.txt', 'w') as f:
    for dialog in sample_dialogs:
        f.write(dialog + '\n')

print("✓ Sample dialog data created")

In [ ]:
# Prepare Cornell Movie dataset helper script
with open('prepare_cornell_data.py', 'w') as f:
    f.write('''import os

def prepare_cornell_data():
    """Convert Cornell Movie dataset to our format"""
    lines_file = 'cornell_movie_dialogs/movie_lines.txt'
    conversations_file = 'cornell_movie_dialogs/movie_conversations.txt'
    output_file = 'data/cornell_conversations.txt'
    
    # Load lines
    lines = {}
    with open(lines_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) >= 5:
                lines[parts[0]] = parts[4].strip()
    
    # Create conversation pairs
    pairs = []
    with open(conversations_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) >= 4:
                conv = eval(parts[3])
                for i in range(len(conv) - 1):
                    if conv[i] in lines and conv[i + 1] in lines:
                        input_text = lines[conv[i]].replace('|', '')
                        output_text = lines[conv[i + 1]].replace('|', '')
                        pairs.append(f"{input_text}|{output_text}")
    
    # Save
    os.makedirs('data', exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        for pair in pairs[:5000]:  # Limit to 5000 for faster training
            f.write(pair + '\\n')
    
    print(f"✓ Prepared {len(pairs[:5000])} conversation pairs")

if __name__ == '__main__':
    prepare_cornell_data()
''')

!python prepare_cornell_data.py

In [ ]:
# List all available data files
!ls -lh conversational_data.txt data/

## 3. Train the Model

Now let's train the model! You can choose between:
- **Transformer** (recommended for better performance)
- **LSTM** (faster training, good for small datasets)

### Option A: Train Transformer Model

In [ ]:
# Train Transformer model on multiple datasets
!python train_pytorch.py \
    --model transformer \
    --data conversational_data.txt data/cornell_conversations.txt \
    --epochs 10 \
    --batch_size 32 \
    --lr 0.001 \
    --d_model 256 \
    --max_len 128 \
    --save_dir models \
    --device auto

### Option B: Train LSTM Model (Faster Alternative)

In [ ]:
# Train LSTM model (faster, lighter)
!python train_pytorch.py \
    --model lstm \
    --data conversational_data.txt data/cornell_conversations.txt \
    --epochs 15 \
    --batch_size 64 \
    --lr 0.001 \
    --d_model 256 \
    --max_len 128 \
    --save_dir models \
    --device auto

## 4. Test the Trained Model

Let's test our trained model with some example inputs.

In [ ]:
# Test with single inputs
test_inputs = [
    "hello",
    "how are you",
    "what is your name",
    "tell me about your company",
    "thank you"
]

for text in test_inputs:
    print(f"\n{'='*60}")
    print(f"Input: {text}")
    !python inference_pytorch.py --model models/best_model.pt --vocab models/vocabulary.json --text "{text}" --temperature 0.8
    print(f"{'='*60}")

## 5. Interactive Chat

Start an interactive chat session with the model.

In [ ]:
# Interactive chat (will work in Colab with input prompts)
from inference_pytorch import ChatBot

# Initialize chatbot
chatbot = ChatBot(
    model_path='models/best_model.pt',
    vocab_path='models/vocabulary.json',
    device='auto'
)

# Chat interactively
print("\nType your message below. Type 'quit' to exit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("Goodbye!")
        break
    
    response = chatbot.generate_response(user_input, temperature=0.8)
    print(f"TheBrain: {response}\n")

## 6. Download Trained Model

Download your trained model to use locally.

In [ ]:
# Zip the models directory
!zip -r trained_model.zip models/

# Download in Colab
from google.colab import files
files.download('trained_model.zip')

print("\n✓ Model downloaded! You can now use it locally.")

## 7. Model Architecture Visualization

Let's visualize the model architecture and training statistics.

In [ ]:
# Model summary
import torch
from pytorch_model import TransformerModel
import json

# Load vocabulary
with open('models/vocabulary.json', 'r') as f:
    vocab_data = json.load(f)

vocab_size = vocab_data['vocab_size']

# Load checkpoint
checkpoint = torch.load('models/best_model.pt', map_location='cpu')

print("Model Information:")
print(f"  - Model Type: {checkpoint.get('model_type', 'transformer')}")
print(f"  - Vocabulary Size: {vocab_size:,}")
print(f"  - Model Dimension: {checkpoint.get('d_model', 256)}")
print(f"  - Best Validation Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
print(f"  - Trained Epochs: {checkpoint.get('epoch', 'N/A') + 1}")

# Count parameters
if checkpoint.get('model_type') == 'transformer':
    model = TransformerModel(vocab_size=vocab_size, d_model=checkpoint.get('d_model', 256))
    model.load_state_dict(checkpoint['model_state_dict'])
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nParameter Count:")
    print(f"  - Total Parameters: {total_params:,}")
    print(f"  - Trainable Parameters: {trainable_params:,}")
    print(f"  - Model Size: ~{total_params * 4 / 1024 / 1024:.2f} MB")

## 8. Custom Dataset Integration

Here's how to add your own dataset:

In [ ]:
# Example: Create custom dataset
custom_data = [
    "What is AI?|Artificial Intelligence is the simulation of human intelligence by machines.",
    "What is machine learning?|Machine learning is a subset of AI that learns from data.",
    "What is deep learning?|Deep learning uses neural networks with multiple layers.",
    "What is PyTorch?|PyTorch is a popular deep learning framework developed by Facebook.",
    "What is a transformer?|Transformer is an attention-based neural network architecture."
]

# Save custom dataset
with open('data/custom_ai_qa.txt', 'w') as f:
    for pair in custom_data:
        f.write(pair + '\n')

print("✓ Custom dataset created!")
print("\nTo train with your custom dataset, add it to the --data argument:")
print("python train_pytorch.py --data conversational_data.txt data/custom_ai_qa.txt ...")

## 9. Tips for Better Results

### Training Tips:
1. **More data = better results**: Combine multiple datasets
2. **Adjust learning rate**: Try 0.0001 to 0.001
3. **Increase model size**: Use `--d_model 512` for better capacity
4. **Train longer**: More epochs often help (20-50)
5. **Use GPU**: Enable GPU runtime in Colab (Runtime → Change runtime type → GPU)

### Inference Tips:
1. **Temperature**: 
   - Lower (0.5-0.7) = more focused, deterministic
   - Higher (0.9-1.2) = more creative, random
2. **Max length**: Adjust based on your use case
3. **Fine-tune**: Continue training on domain-specific data

### Common Issues:
1. **Out of memory**: Reduce `--batch_size` or `--d_model`
2. **Poor responses**: Need more training data or epochs
3. **Repetitive outputs**: Adjust temperature or use beam search

## 10. Next Steps

Congratulations! You've successfully trained and deployed TheBrain AI model. Here are some next steps:

1. **Improve the model**:
   - Add more training data
   - Increase model size
   - Try different architectures

2. **Deploy the model**:
   - Create a web API with Flask/FastAPI
   - Build a chatbot interface
   - Integrate into your application

3. **Experiment**:
   - Try different hyperparameters
   - Test on different domains
   - Implement beam search for better generation

### Resources:
- PyTorch Documentation: https://pytorch.org/docs/
- Transformer Paper: "Attention Is All You Need"
- TheBrain Repository: https://github.com/meahmedrabbi/TheBrain

---

**Happy Training! 🚀**